In [90]:
import pandas as pd
import numpy as np
from surprise.model_selection import KFold
import json

## Obtain Data

In [22]:
reviews_df = pd.read_csv('shared_final_prediction_file.csv')

In [78]:
reviews_df.describe()

,rowid,listing_id,reviewer_id,polarity
count,1.053172e+06,1.053172e+06,1.053172e+06,1.053172e+06
mean,5.265855e+05,1.131222e+07,6.676650e+07,8.133919e-01
std,3.040247e+05,8.636989e+06,6.249576e+07,2.722260e-01
min,0.000000e+00,2.539000e+03,1.000000e+00,-1.000000e+00
25%,2.632928e+05,3.394964e+06,1.503578e+07,7.906000e-01
50%,5.265855e+05,1.005259e+07,4.500549e+07,9.215000e-01
75%,7.898782e+05,1.839986e+07,1.069479e+08,9.666667e-01
max,1.053171e+06,3.277125e+07,2.470066e+08,1.000000e+00


In [24]:
reviews_df.shape

(1053172, 4)

In [25]:
reviews_df.polarity.describe()

count    1.053172e+06
mean     8.133919e-01
std      2.722260e-01
min     -1.000000e+00
25%      7.906000e-01
50%      9.215000e-01
75%      9.666667e-01
max      1.000000e+00
Name: polarity, dtype: float64

In [26]:
from surprise import SVD, KNNBasic, Dataset, Reader, accuracy
from surprise.model_selection import cross_validate, train_test_split

In [27]:
reader = Reader(rating_scale=(-1,1))

In [28]:
data = Dataset.load_from_df(reviews_df[['listing_id', 'reviewer_id', 'polarity']], reader)

In [36]:
trainset, testset = train_test_split(data, test_size=.3)

In [37]:
algo = SVD(n_factors=10, n_epochs=10)

In [38]:
# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)

In [39]:
predictions = algo.test(testset)

In [40]:
accuracy.rmse(predictions, verbose=True)

RMSE: 0.2659


0.26592393666752945

In [43]:
# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 0.2661


0.2660984221479662

In [44]:
p = predictions[0]

In [45]:
p

Prediction(uid=21065293, iid=20810098, r_ui=0.7424, est=0.7428243756684929, details={'was_impossible': False})

In [48]:
output_file = open("/Users/vishu/SJSU/cmpe256/AirbnbProject/cmpe256-airbnbproject/svd_predictions.csv", "w")
output_file.write("userid,itemid,rating\n".format(user,item,rating ))

for i in range(0, len(predictions)):
    pred_tup = predictions[i]
    user = pred_tup[0]
    item = pred_tup[1]
    rating = pred_tup[3]
    output_file.write("{},{},{}\n".format(user,item,rating ))

output_file.close()

### Using SVDpp

In [13]:
from surprise import SVDpp

In [15]:
#SVD Matrix Factorization
algo = SVDpp(n_factors=10,n_epochs=10)
algo.fit(trainset)
predictions_svd = algo.test(testset)

In [16]:
accuracy.rmse(predictions_svd)

RMSE: 0.2669


0.26688308324524357

### Knn

In [12]:
#lets configure some parameters for Collaborative Filtering Algo
sim_options = {
    'name': 'pearson', #similarity measure
    'user_based': False #item based
}
#For user based ->True
#For name ->pearson,cosine,msd,pearson_baseline


In [ ]:
#this is baseline configuration for optimizing the error
bsl_options = {'method': 'als',#another option is sgd
               'n_epochs': 5, # number of iterations
               'reg_u': 12, # user-regularisation parameter
               'reg_i': 5   # item-regularisation parameter
               }

We will try KNN with maximum neighbour as 3 and minimum as 1.We train our model on trainingset and generate prediction on test set.

In [13]:
#KNN
knn = KNNBasic(sim_options=sim_options,k=3,min_k=1) #neighbours=3


In [ ]:
knn.fit(trainset)


Computing the pearson similarity matrix...


In [ ]:
predictions_knn = knn.test(testset)

## Predicting top 10 recomendations

In [79]:
from collections import defaultdict
 
def get_top10_recommendations(predictions, topN = 10):
     
    top_recs = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_recs[uid].append((iid, est))
     
    for uid, user_ratings in top_recs.items():
        user_ratings.sort(key = lambda x: x[1], reverse = True)
        top_recs[uid] = user_ratings[:topN]
     
    return top_recs

In [80]:
top10_recommendations = get_top10_recommendations(predictions)

In [92]:
# For each user we print top 10 recommendations
final_reco = dict()

for uid, user_ratings in top10_recommendations.items():
    final_reco[uid] = [iid for (iid, _) in user_ratings]
    
with open('svd_user_recommendations.json', 'w') as fp:
    json.dump(final_reco, fp)

fp.close()